In [1]:
import os 
import pandas as pd 
import numpy  as np
import matplotlib.pyplot as plt 

In [6]:
data_dir="E:\\doppnet_for_classification"

In [7]:
import torch
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [ ]:
transform = transforms.Compose([
    transforms.Resize((128, 128)),   
    transforms.ToTensor()            
])

In [ ]:
dataset = datasets.ImageFolder(root=data_dir, transform=transform)

train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, 3, padding=1)
        self.conv2 = nn.Conv2d(16, 32, 3, padding=1)
        self.conv3 = nn.Conv2d(32, 64, 3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(64 * 16 * 16, 128)  
        self.fc2 = nn.Linear(128, 4)  

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = x.view(-1, 64 * 16 * 16)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x


In [ ]:
if torch.cuda.is_available:
    device=torch.device("cuda")

import torch.optim as optim

In [14]:
model = SimpleCNN()
model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
import torch.optim as optim

losses=[]

num_epochs = 10
for epoch in range(num_epochs):
    running_loss = 0.0
    for images, labels in train_loader:
        images,labels=images.to(device),labels.to(device)
        optimizer.zero_grad()      
        outputs = model(images)    
        loss = criterion(outputs, labels)  
        loss.backward()            
        optimizer.step()            

        running_loss += loss.item()
        losses.append(loss.item())

    print(f"Epoch {epoch+1}, Loss: {running_loss/len(train_loader):.4f}")

print("Eğitim tamamlandı")


Epoch 1, Loss: 1.3293
Epoch 2, Loss: 0.6272
Epoch 3, Loss: 0.3904
Epoch 4, Loss: 0.2219
Epoch 5, Loss: 0.1644
Epoch 6, Loss: 0.1086
Epoch 7, Loss: 0.0708
Epoch 8, Loss: 0.0536
Epoch 9, Loss: 0.0268
Epoch 10, Loss: 0.0429
Eğitim tamamlandı!


In [16]:
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        images,labels=images.to(device),labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Test Accuracy: {100 * correct / total:.2f}%")


Test Accuracy: 96.10%


In [17]:
model_save_path = "model_dopnet_cnn2d.pth"  # Model kaydetme yolu ve dosya adı
torch.save(model.state_dict(), model_save_path)
print(f"Model saved to {model_save_path}")

Model saved to model_dopnet_cnn2d.pth
